In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: a17b7aca-5552-4f17-9ba0-9946029390a6: POC-Projeto APR>, <Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: 4a5e0119-fb85-40fd-87ab-f201a2aa0fde: GSB - UHE-MACHADINHO>, <Company: 7fe7b473-6d9c-49d6-a9ca-74f3424d3211: GSB - UHE-SALTO OSORIO>, <Company: ed749424-27e4-4342-82cd-f924de71830b: Limbo - UTPS>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: b75339ff-e91f-4e75-a082-5d7e23a4d049: GSB - UHE-SALTO SANTIAGO>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: d25d0eb2-61a8-4646-8e0e-31c8e68cf216: TESTE>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b

In [11]:
from helpers.signals import DisableSignals
from tqdm.notebook import tqdm
import uuid

In [3]:
company_origin=Company.objects.get(name="CCR - Rio Sp")

In [4]:
User.objects.all().count()

68

User manipulation

In [5]:
company_name = "CCR - RodoAnel"

In [53]:
key_user = User.objects.get(username="marcos")

In [54]:
company = Company(name=company_name,
                  company_group=company_origin.company_group,
                 cnpj=company_origin.cnpj,
                 custom_options = company_origin.custom_options,
                 metadata = company_origin.metadata)

In [55]:
# company.company_group = None
company.save()

In [6]:
company=Company.objects.get(name=company_name)
company

<Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>

In [57]:
# users_to_transfer = User.objects.filter(companies=company)
users_to_transfer = User.objects.filter(username__in=[
    "felippe.vanroo",
    "luan.poletti",
    "pedro.gasparotti",
    "marina.carpes"
])

In [58]:
for a in users_to_transfer:
    a.company_group_id = company_origin.company_group.uuid
    a.save()

In [59]:
for a in SubCompany.objects.filter(company=company_origin):
    a.uuid=None
    a.company=company
    a.save()

In [60]:
for a in UserPermission.objects.filter(companies=company_origin):
    a.companies.add(company)

In [61]:
for a in Firm.objects.filter(company=company_origin):
    a.uuid=None
    a.company = company
    a.subcompany= SubCompany.objects.filter(company=company,name = a.subcompany.name).first()
    a.save()

In [62]:
for a in UserInFirm.objects.filter(firm__company=company_origin):
    a.uuid=None
    a.firm=Firm.objects.get(name=a.firm.name,company=company)
    a.save()

In [63]:
for a in UserInCompany.objects.filter(company=company_origin):
    a.uuid = None
    a.company = company
    a.save()

In [64]:
for a in ServiceOrderActionStatusSpecs.objects.filter(company=company_origin):
    a.uuid=None
    a.company=company
    a.save()

In [7]:
for a in OccurrenceTypeSpecs.objects.filter(company=company_origin):
    a.uuid=None
    a.company = company
    a.save()

In [ ]:
# road_ids = Road.objects.filter(company=company).values_list("id", flat=True)

In [ ]:
# for road_id in road_ids:
#     this_road = Road.objects.get(id=road_id)
#     print(this_road)
#     with DisableSignals():
#         this_road.save(using="ccr_homolog")
#     this_road.company.add(company)

In [66]:
for a in ShapeFile.objects.filter(companies=company_origin):
    a.companies.add(company)

In [67]:
for a in TileLayer.objects.filter(companies=company_origin):
    a.companies.add(company)

In [68]:
for a in ApprovalFlow.objects.filter(company=company_origin):
    a.uuid = None
    a.company=company
    a.save()

In [69]:
for a in ApprovalStep.objects.filter(approval_flow__company=company_origin):
    a.uuid=None
    a.approval_flow = ApprovalFlow.objects.get(name=a.approval_flow.name,company=company)
    a.save()

In [70]:
for a in ApprovalTransition.objects.filter(origin__approval_flow__company=company_origin):
    a.uuid=None
    a.origin = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.origin.name,
                                        approval_flow__target_model=a.origin.approval_flow.target_model)
    a.destination = ApprovalStep.objects.get(approval_flow__company=company,
                                        name=a.destination.name,
                                        approval_flow__target_model=a.destination.approval_flow.target_model)
    a.save()
    print(a.origin,a.destination)

[CCR SPVias]: RDO Aguardando Aprovação [CCR SPVias]: RDO Aprovado
[CCR SPVias]: Aguardando Aprovação [CCR SPVias]: Cancelado
[CCR SPVias]: Aguardando Aprovação [CCR SPVias]: Aprovado
[CCR SPVias]: Aguardando Aprovação [CCR SPVias]: Em Revisão
[CCR SPVias]: Criado [CCR SPVias]: Aguardando Aprovação
[CCR SPVias]: RDO Criado [CCR SPVias]: RDO Aguardando Aprovação
[CCR SPVias]: Aguardando Aprovação [CCR SPVias]: Aprovado
[CCR SPVias]: Em Revisão [CCR SPVias]: Aguardando Aprovação


# Opcional necessita de ajustes

In [ ]:
for a in tqdm(
    Reporting.objects.filter(
        Q(uuid__in=Job.objects.filter(company=company_origin, inspection__isnull=False).values_list("inspection_id", flat=True).distinct())
        | Q(uuid__in=Job.objects.filter(company=company_origin, parent_inventory__isnull=False).values_list("parent_inventory_id", flat=True).distinct())
    )
):
    if Reporting.objects.using("ccr_homolog").filter(uuid=a.uuid).exists():
        continue
    a.active_inspection = None
    a.parent = None
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(
    Job.objects.filter(company=company)
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(Job.history.model.objects.filter(company=company)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(Reporting.objects.filter(company=company)):
    a.active_inspection = None
    a.parent = None
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(Reporting.objects.filter(company=company, parent__isnull=False)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(Reporting.objects.filter(company=company, active_inspection__isnull=False)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(Reporting.history.model.objects.filter(company=company)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(ReportingFile.objects.filter(reporting__company=company)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in tqdm(ReportingFile.history.model.objects.filter(reporting__company=company)):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [6]:
for a in tqdm(ReportingFile.objects.filter(reporting__company=company)):
    print(a.upload)

  0%|          | 0/158 [00:00<?, ?it/s]

DM1_400x300_aJw6kIA.jpg
DM1_400x300_AfqsvBN.jpg
oacinsp_Ve5VJLX.jpg
tout_court_51ZfjKh.jpeg
acessosRecup_DMfWojR.jpg
acessos2_Ytqrr1x.jpg
recupTalude_sOV7NNz.jpg
DSC_0027.JPG
MF_101_RS_066890_S_1_-_OK.jpeg
SJ_101_RS_023290_S1_-_Não_localizado.jpeg
DI_101_RS_028781_S_1_-_OK.jpeg
0190c80c-df9c-4b23-b93a-872d938e5698-0_vC3CYxi.jpeg
Exportação_ANTT_1_1.png
Dúvidas_-_Relatórios_Pavimentos.docx
edificacoes_922sbNX.jpeg
postes.jpg
edificacoes_xlbNzJd.jpeg
edificacoes_0bHYjoe.jpeg
oacinsp_goCQB4r.jpg
images_X9jh3Dz.jpg
drenagem-profunda-onde-aplicar_TqcNJOm.jpg
drenagem-profunda-onde-aplicar_QLDxrmO.jpg
drenagem-profunda-onde-aplicar.jpg
drenagem-profunda-onde-aplicar.jpg
drenagem-profunda-onde-aplicar.jpg
acessosRecup_J4tgUEG.jpg
7ca44d77-46ce-4520-b575-aa6d4e99550a-0_g76i11V.jpeg
TESTE_2.PNG
TESTE_1.PNG
drenagemsup_vTYObUF.jpeg
DM1_400x300_EDbSKk9.jpg
acessos2_fYH1ks6.jpg
instalacao_vzUYrel.png
drenagem-profunda-onde-aplicar_m12ENXC.jpg
drenagem-profunda-onde-aplicar_How5Nfo.jpg
drenagem-pro

In [ ]:
for a in AccessRequest.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in AccessRequest.history.model.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in ContractService.objects.filter(
    Q(administration_service_contracts__subcompany__company=company) |
    Q(unit_price_service_contracts__subcompany__company=company) |
    Q(performance_service_contracts__subcompany__company=company)
):
#     with DisableSignals():
#         a.save(using="ccr_homolog")
    new_obj = ContractService.objects.using("ccr_homolog").get(uuid=a.uuid)
    new_obj.contract_item_unit_prices.set([])
    new_obj.contract_item_administration.set([])
    new_obj.contract_item_performance.set([])

    for service in a.contract_item_unit_prices.all():
        new_service = ContractItemUnitPrice.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.contract_item_unit_prices.add(new_service)
    for service in a.contract_item_administration.all():
        new_service = ContractItemAdministration.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.contract_item_administration.add(new_service)
    for service in a.contract_item_performance.all():
        new_service = ContractItemPerformance.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.contract_item_performance.add(new_service)
    for firm in a.firms.all():
        new_firm = Firm.objects.using("ccr_homolog").get(uuid=firm.uuid)
        new_obj.firms.add(new_firm)
    

In [ ]:
for a in Contract.objects.filter(
    subcompany__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")
    new_obj = Contract.objects.using("ccr_homolog").get(uuid=a.uuid)
    for service in a.unit_price_services.all():
        new_service = ContractService.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.unit_price_services.add(new_service)
    for service in a.administration_services.all():
        new_service = ContractService.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.administration_services.add(new_service)
    for service in a.performance_services.all():
        new_service = ContractService.objects.using("ccr_homolog").get(uuid=service.uuid)
        new_obj.performance_services.add(new_service)

In [ ]:
for a in Resource.objects.filter(
    company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in AdditionalControl.objects.filter(
    company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in ServiceOrderResource.objects.filter(
    resource__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in ContractItemAdministration.objects.filter(
    contract_item_administration_services__administration_service_contracts__subcompany__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in ContractItemUnitPrice.objects.filter(
    contract_item_unit_price_services__unit_price_service_contracts__subcompany__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in ContractItemPerformance.objects.filter(
    contract_item_performance_services__performance_service_contracts__subcompany__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in MeasurementBulletin.objects.filter(
    contract__subcompany__company=company
):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in MultipleDailyReport.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportWorker.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportEquipment.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportVehicle.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportSignaling.objects.filter(company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportOccurrence.objects.filter(multiple_daily_reports__company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
for a in DailyReportRelation.objects.filter(daily_report__company=company):
    with DisableSignals():
        a.save(using="ccr_homolog")

In [ ]:
from django.utils.crypto import get_random_string
def generate_secret_key(length=50):
    """
    Return a 50 character random string
    usable as a `SECRET_KEY` setting value.
    """
    chars = 'abcdefghijklmnopqrstuvwxyz0123456789!@#$%^&*(-_=+)'
    if not isinstance(length, int):
        raise TypeError(
            f'invalid literal for int() with base 10: {length}')
    return get_random_string(length, chars)

In [ ]:
for a in User.objects.using("ccr_homolog").filter(companies=company):
    a.jwt_secret = generate_secret_key()
    a.save()

In [ ]:
for a in User.objects.using("ccr_homolog").filter(companies=company, email__icontains="grupoccr.com.br"):
    a.saml_idp = "https://sts.windows.net/d233d58a-9973-43a7-af69-6763630548a0/"
    a.saml_nameid = a.email
    a.save()

In [ ]:
for a in User.objects.filter(companies=company, email__icontains="grupoccr.com.br"):
    a.auth_error = "kartado.infra.redirectEnv[https://homolog-ccr.kartado.com.br]"
    a.save()

In [ ]:
with DisableSignals():
    Reporting.objects.using("ccr_homolog").filter(company=company, occurrence_type__isnull=True).delete()

In [ ]:
LogEntry.objects.using("ccr_homolog").delete()

In [11]:
occs=OccurrenceType.objects.filter(updated_at__gte='2023-8-7')
occs.count(),occs[0]

(1,
 <OccurrenceType: Reconstrução em Edificações - ['CCR - AutoBAn', 'CCR - RodoAnel', 'CCR - Rio Sp', 'CCR - SPVias']>)

In [33]:
from tqdm.notebook import tqdm
for a in tqdm(OccurrenceTypeSpecs.objects.filter(company=company)):
    a.delete()

  0%|          | 0/238 [00:00<?, ?it/s]

In [32]:
OccurrenceTypeSpecs.objects.filter(company=company).count()

238